In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from amplpy import AMPL, ampl_notebook
from joblib import load, dump

pd.set_option("future.no_silent_downcasting", True) # Prevent silent data type changes during operations for future compatibility

Instantiate the AMPL object

In [12]:
ampl = ampl_notebook(
    modules=["highs","coin"], # solver to be used
    license_uuid="default" # license key
)

In [7]:
%%ampl_eval
reset;

In [8]:
%%ampl_eval
# Sets
set ROUTES;
set AIRPLANE_TYPES;

# Parameters
param a {ROUTES};          # Base fare price coefficient
param b {ROUTES};          # Price elasticity coefficient
param r {ROUTES};          # Yearly effect coefficient
param d {ROUTES};          # Quarter 1 effect coefficient
param e {ROUTES};          # Quarter 2 effect coefficient
param z {ROUTES};          # Quarter 3 effect coefficient
param h {ROUTES};          # Quarter 4 effect coefficient
param c {ROUTES};          # Cost per seat
param N {ROUTES};          # Predicted daily passengers
param C {AIRPLANE_TYPES};  # Capacity of each airplane type
param R {AIRPLANE_TYPES};  # Total airplanes of each type
param t;                   # Target market share (fraction)
param year;                # Current year
param active_quarter;      # Active quarter (1, 2, 3, or 4)

# Variables
var x {ROUTES} >= 0, integer;      # Daily number of seats sold
var r_assign {ROUTES, AIRPLANE_TYPES} >= 0, integer;  # Number of airplanes assigned per route
var q {1..4} binary;               # Quarter indicator variables

# Objective
maximize Profit:
    sum {i in ROUTES} (
        x[i] * (
            a[i] * exp(-b[i] * x[i] + r[i] * year +
                       d[i] * q[1] + e[i] * q[2] +
                       z[i] * q[3] + h[i] * q[4]
            ) - c[i]
        )
    );

# Constraints
s.t. MarketShare {i in ROUTES}:
    x[i] <= t * N[i];

s.t. MinCapacity {i in ROUTES}:
    x[i] >= sum {k in AIRPLANE_TYPES} 0.9 * C[k] * r_assign[i, k];

s.t. MaxCapacity {i in ROUTES}:
    x[i] <= sum {k in AIRPLANE_TYPES} C[k] * r_assign[i, k];

s.t. FleetLimit {k in AIRPLANE_TYPES}:
    sum {i in ROUTES} r_assign[i, k] <= 3 * R[k];

s.t. ActiveQuarter:
    sum {qtr in 1..4} q[qtr] = 1;

s.t. QuarterActive:
    q[active_quarter] = 1;

In [9]:
# Define data for the problem
routes = ['AUS_IAD', 'LAX_ORD', 'JFK_LHR', 'SEA_SFO']
airplane_types = ['type1', 'type2']

# Parameter values
a = {'AUS_IAD': 100, 'LAX_ORD': 120, 'JFK_LHR': 150, 'SEA_SFO': 90}
b = {'AUS_IAD': 0.01, 'LAX_ORD': 0.02, 'JFK_LHR': 0.03, 'SEA_SFO': 0.015}
r = {'AUS_IAD': 0.1, 'LAX_ORD': 0.15, 'JFK_LHR': 0.2, 'SEA_SFO': 0.12}
d = {'AUS_IAD': 0.05, 'LAX_ORD': 0.06, 'JFK_LHR': 0.07, 'SEA_SFO': 0.04}
e = {'AUS_IAD': 0.04, 'LAX_ORD': 0.05, 'JFK_LHR': 0.06, 'SEA_SFO': 0.03}
z = {'AUS_IAD': 0.03, 'LAX_ORD': 0.04, 'JFK_LHR': 0.05, 'SEA_SFO': 0.025}
h = {'AUS_IAD': 0.02, 'LAX_ORD': 0.03, 'JFK_LHR': 0.04, 'SEA_SFO': 0.02}
c = {'AUS_IAD': 50, 'LAX_ORD': 60, 'JFK_LHR': 70, 'SEA_SFO': 45}
N = {'AUS_IAD': 200, 'LAX_ORD': 250, 'JFK_LHR': 300, 'SEA_SFO': 180}
C = {'type1': 150, 'type2': 200}
R = {'type1': 5, 'type2': 3}

# Load data into AMPL
ampl.set['ROUTES'] = routes
ampl.set['AIRPLANE_TYPES'] = airplane_types
ampl.param['a'] = a
ampl.param['b'] = b
ampl.param['r'] = r
ampl.param['d'] = d
ampl.param['e'] = e
ampl.param['z'] = z
ampl.param['h'] = h
ampl.param['c'] = c
ampl.param['N'] = N
ampl.param['C'] = C
ampl.param['R'] = R
ampl.param['t'] = 0.3
ampl.param['year'] = 2024
ampl.param['active_quarter'] = 3

In [10]:
%%ampl_eval
option solver ipopt;
solve;

Solution determined by presolve;
objective Profit = 0.


In [11]:
# Retrieve results
x = ampl.getVariable("x").getValues().toPandas()
r_assign = ampl.getVariable("r_assign").getValues().toPandas()
profit = ampl.getObjective("Profit").value()

# Display results
print("Optimal Seat Sales (x):")
print(x)

print("\nAirplane Assignments (r_assign):")
print(r_assign)

print(f"\nTotal Profit: {profit}")

Optimal Seat Sales (x):
         x.val
AUS_IAD      0
JFK_LHR      0
LAX_ORD      0
SEA_SFO      0

Airplane Assignments (r_assign):
                r_assign.val
index0  index1              
AUS_IAD type1              0
        type2              0
JFK_LHR type1              0
        type2              0
LAX_ORD type1              0
        type2              0
SEA_SFO type1              0
        type2              0

Total Profit: 0.0
